$\Large\textbf{Lab 2. Exercise 2. }$



Recall that we implemented the gradient descent algorithm to solve $\min_{\mathbf{x} \in {\mathbb{R}}^n} f(\mathbf{x})$. The main ingredients in the gradient descent iterations are the descent direction $\mathbf{p}^k$ which is set to $-\nabla f(\mathbf{x}^k)$, and the step length $\eta^k$ which is found by solving an optimization problem (or sometimes taken as a constant value over all iterations). We used the following procedure in the previous lab:

\begin{align}
& \textbf{Input:} \text{ Starting point $x^0$, Stopping tolerance $\tau$}  \\
& \textbf{Initialize } k=0 \\ 
& \mathbf{p}^k =-\nabla f(\mathbf{x}^k) \\ 
&\textbf{While } \| \mathbf{p}^k \|_2 > \tau \text{ do:}  \\   
&\quad \quad \eta^k = \arg\min_{\eta\geq 0} f(\mathbf{x}^k + \eta  \mathbf{p}^k) = \arg\min_{\eta\geq 0} f(\mathbf{x}^k - \eta  \nabla f(\mathbf{x}^k)) \\
&\quad \quad \mathbf{x}^{k+1} = \mathbf{x}^k + \eta^k \mathbf{p}^k = \mathbf{x}^k - \eta^k \nabla f (\mathbf{x}^k)  \\ 
&\quad \quad k = {k+1} \\ 
&\textbf{End While} \\
&\textbf{Output: } \mathbf{x}^k
\end{align}

We saw that for quadratic functions, a closed form analytical solution for the minimizer of the optimization problem $\min_{\eta \geq 0} f({\mathbf{x}}^k + \eta {\mathbf{p}}^k)$ exists. However finding a closed form expression as a solution to this optimization problem to find a suitable step length might not always be possible. To tackle general situations, we will try to devise a different procedure in this lab. 

To find the step length, we will use the following property: 
Suppose a non-zero $\mathbf{p} \in {\mathbb{R}}^n$ is a descent direction at point $\mathbf{x}$, and let $\gamma \in (0,1)$. Then there exists $\varepsilon >0$ such that  
\begin{align}
f(\mathbf{x}+\alpha \mathbf{p}) \leq f(\mathbf{x}) + \gamma \alpha \nabla f(\mathbf{x})^\top \mathbf{p}, \ \forall \alpha \in (0,\varepsilon].  
\end{align}

The step length $\eta^k$ can be found using a backtracking procedure illustrated below to find appropriate value of $\varepsilon$.  




\begin{align}
& \textbf{Input:}  \text{ $\mathbf{x}^k$, $\mathbf{p}^k$, $\alpha^0$, $\rho \in (0,1)$, $\gamma \in (0,1)$ }  \\
& \textbf{Initialize } \alpha=\alpha^0 \\ 
&\textbf{While } f(\mathbf{x}^k + \alpha \mathbf{p}^k)   > f(\mathbf{x}^k) + \gamma \alpha \nabla f(\mathbf{x}^k)^\top \mathbf{p}^k \text{ do:}  \\   
&\quad \quad \alpha = \rho \alpha  \\
&\textbf{End While} \\
&\textbf{Output: } \alpha
\end{align}

In this exercise, we will check if finding the steplength using the backtracking procedure is advantageous for some quadratic functions. In this sample code we consider $h(\textbf{x}) = h(x_1, x_2, x_3) = \frac{1}{8}(x_1 - 2)^2 + \frac{1}{64}(x_2 - 4)^2 + \frac{1}{512}(x_3 - 8)^2  $.

In [34]:
#numpy package will be used for most of our lab exercises. Please have a look at https://numpy.org/doc/1.19/ for numpy documentation
#we will first import the numpy package and name it as np
import numpy as np 
import matplotlib.pyplot as plt
#Henceforth, we can lazily use np to denote the much longer numpy !! 

In [35]:
#Now we will define a function which will compute and return the function value 
def evalf(x):  
  #Input: x is a numpy array of size 3 
  assert type(x) is np.ndarray and len(x) == 3 #do not allow arbitrary arguments 
  #after checking if the argument is valid, we can compute the objective function value
  #compute the function value and return it 
  a = 0
  for i in range(1, len(x)+ 1):
    a = a + 8**(-i) * (x[i-1]- 2**i)**2
  return a

In [36]:
#Now we will define a function which will compute and return the gradient value as a numpy array 
def evalg(x):  
  #Input: x is a numpy array of size 3
  assert type(x) is np.ndarray and len(x) == 3 #do not allow arbitrary arguments 
  #after checking if the argument is valid, we can compute the gradient value
  #compute the gradient value and return it 
  y = []
  for i in range(1, len(x)+1):
    y.append(2 * 8**(-i) * (x[i-1]- 2**i))
  return np.array(y)

In [37]:
#Complete the module to compute the steplength by using the closed-form expression
def compute_steplength_exact(gradf, A): #add appropriate arguments to the function 
  #assert type(gradf) is np.ndarray and len(gradf) == 3
  #assert type(A) is np.ndarray and A.shape[0] == 3 and  A.shape[1] == 3 #allow only a 2x2 array
   
  #Complete the code to compute step length
  g = gradf
  g_t = np.matrix.transpose(g)
  step_length = np.matmul(g_t, g)/(2*np.matmul(np.matmul(g_t, A), g))
  
  return step_length

In [38]:
#Complete the module to compute the steplength by using the backtracking line search
def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function 
  #assert type(x) is np.ndarray and len(gradf) == 3
  #assert type(gradf) is np.ndarray and len(gradf) == 3
  
  alpha = alpha_start
  p = - gradf
  #implement the backtracking line search
  #print(evalf(x + alpha*p), evalf(x) + gamma * alpha* (np.matmul(np.matrix.transpose(p), p) ))
  while evalf(x + alpha*p) > evalf(x) + gamma * alpha* (np.matmul(np.matrix.transpose(-p), p) ):
    alpha = alpha * rho
  #print('final step length:',alpha)
  return alpha

In [39]:
#we define the types of line search methods that we have implemented
EXACT_LINE_SEARCH = 1
BACKTRACKING_LINE_SEARCH = 2
CONSTANT_STEP_LENGTH = 3

In [40]:
def find_minimizer(start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size 3, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 3 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  # construct a suitable A matrix for the quadratic function 
  A = np.array([[1/8, 0, 0], [0, 1/64, 0], [0, 0, 1/512]])
  x = start_x
  g_x = evalg(x)

  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    #print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)

  k = 0
  #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
  
    if line_search_type == EXACT_LINE_SEARCH:
      step_length = compute_steplength_exact(g_x, A) #call the new function you wrote to compute the steplength
      #raise ValueError('EXACT LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking(x, g_x, alpha_start, rho, gamma) #call the new function you wrote to compute the steplength
      #raise ValueError('BACKTRACKING LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 0.1
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here   
    x = np.subtract(x, np.multiply(step_length,g_x)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point

    #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x, k


In [41]:
my_start_x = np.array([1/64, 1/8, 1])
my_tol= 1e-15

#x_opt, k = find_minimizer(my_start_x, my_tol, CONSTANT_STEP_LENGTH)
#print("Minimizer: {}, Minimum function value: {}".format(x_opt, evalf(x_opt)), k)

print("Using Exact Line Search")
x_opt, k = find_minimizer(my_start_x, 1e-20, EXACT_LINE_SEARCH)
print("Minimizer: {}, Minimum function value: {}".format((x_opt[0], x_opt[1], x_opt[2]), evalf(x_opt)))

print("\nUsing Backtracking Line Search")
#check what happens when you call find_minimzer using backtracking line search
x_opt_bls, k = find_minimizer(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, 1, 0.5,0.5)
print("Minimizer: {}, Minimum function value: {}".format((x_opt_bls[0], x_opt_bls[1], x_opt_bls[2]), evalf(x_opt_bls)))


Using Exact Line Search
Minimizer: (2.0, 4.0, 8.0), Minimum function value: 0.0

Using Backtracking Line Search
Minimizer: (1.9999999999999996, 3.999999999999993, 7.999999999999752), Minimum function value: 1.2074656304934663e-28


$\textbf{Question 2}: $
$h(\textbf{x}) = h(x_1, x_2, x_3) = \frac{1}{8}(x_1 - 2)^2 + \frac{1}{64}(x_2 - 4)^2 + \frac{1}{512}(x_3 - 8)^2  $

We found the minimizer and the munimum function value of $h(\mathbf{x})$ and they are as follows:
*   Minimizer: $(2, 4, 8)$
*   Minimum function value: $0$

$\textbf{Question 3}: $
$h(\textbf{x}) = h(x_1, x_2, x_3) = \frac{1}{8}(x_1 - 2)^2 + \frac{1}{64}(x_2 - 4)^2 + \frac{1}{512}(x_3 - 8)^2  $

We use stopping tolerance $\tau = 10^{-10}$ and the starting point $\textbf{x}^0 = (\frac{1}{64}, \frac{1}{8} , 1)$. We use gradient descent procedure which uses exact step length computation against the
gradient descent procedure which uses the backtracking line search procedure (with $\alpha^0 = 1, \rho = 0.5, \gamma = 0.5$).

In [42]:
my_start_x = np.array([1/64, 1/8, 1])
my_tol= 1e-10

#x_opt, k = find_minimizer(my_start_x, my_tol, CONSTANT_STEP_LENGTH)
#print("Minimizer: {}, Minimum function value: {}".format(x_opt, evalf(x_opt)), k)

print("Using Exact Line Search")
x_opt, k = find_minimizer(my_start_x, my_tol, EXACT_LINE_SEARCH)
print("Minimizer: {}, Minimum function value: {}".format((x_opt[0], x_opt[1], x_opt[2]), evalf(x_opt)), "No. of iterations:", k)

print("\nUsing Backtracking Line Search")
#check what happens when you call find_minimzer using backtracking line search
x_opt_bls, k = find_minimizer(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, 1, 0.5,0.5)
print("Minimizer: {}, Minimum function value: {}".format((x_opt_bls[0], x_opt_bls[1], x_opt_bls[2]), evalf(x_opt_bls)), "No. of iterations:", k)

Using Exact Line Search
Minimizer: (2.0000000001221636, 4.0, 7.999999978377592), Minimum function value: 9.150071377581033e-19 No. of iterations: 269

Using Backtracking Line Search
Minimizer: (1.9999999999999996, 3.999999999999993, 7.999999974451478), Minimum function value: 1.2748574165464873e-18 No. of iterations: 4964




*   Minimizer: (2.0000000001221636, 4.0, 7.999999978377592), 
*   Minimum function value: 9.150071377581033e-19


In gradient descent procedure which uses exact step length computation 
algorithm converged within $269$ iterations while in the gradient descent procedure which uses the backtracking line search procedure (with $\alpha^0 = 1, \rho = 0.5, \gamma = 0.5$) takes $4964$ iterations to converge. Here in this example, exact line search is pretty fast as compared to backtracking line search.

This is because that in the exact line search, we are optimizing line search and finding $\alpha$ which minimizes function in that perticular direction. Hence, gradient descent is faster than backtracking line search.

$\textbf{Question 4}: $
$f(\textbf{x}) = f(x_1, x_2, x_3, x_4) = \frac{1}{8}(x_1 - 2)^2 + \frac{1}{64}(x_2 - 4)^2 + \frac{1}{512}(x_3 - 8)^2 + \frac{1}{4096}(x_4 - 16)^2  $

We choose $\tau = 10^{-10}$ and strating point $\textbf{x}^0 = (\frac{1}{512},\frac{1}{64}, \frac{1}{8} , 1)$. We use gradient descent procedure which uses exact step length computation against the gradient descent procedure which uses the backtracking line search procedure (with $\alpha^0 = 1,  \rho = 0.5, 
 \gamma = 0.5$).

In [43]:
#Now we will define a function which will compute and return the function value 
def evalh(x):  
  a = 0
  for i in range(1, len(x)+ 1):
    a = a + 8**(-i) * (x[i-1]- 2**i)**2
  return a

#Now we will define a function which will compute and return the gradient value as a numpy array 
def evalgh(x):  
  y = []
  for i in range(1, len(x)+1):
    y.append(2 * 8**(-i) * (x[i-1]- 2**i))
  return np.array(y)

def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function  
  alpha = alpha_start
  p = - gradf
  #implement the backtracking line search
  #print(evalf(x + alpha*p), evalf(x) + gamma * alpha* (np.matmul(np.matrix.transpose(p), p) ))
  while evalh(x + alpha*p) > evalh(x) + gamma * alpha* (np.matmul(np.matrix.transpose(-p), p) ):
    alpha = alpha * rho
  #print('final step length:',alpha)
  return alpha

def find_minimizer_for_four_vars(start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size 4, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 4 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  # construct a suitable A matrix for the quadratic function 
  A = np.array([[1/8, 0, 0, 0], [0, 1/64, 0, 0], [0, 0, 1/512, 0], [0, 0, 0, 1/4096]])
  x = start_x
  g_x = evalgh(x)

  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    #print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)

  k = 0
  #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
  
    if line_search_type == EXACT_LINE_SEARCH:
      step_length = compute_steplength_exact(g_x, A) #call the new function you wrote to compute the steplength
      #raise ValueError('EXACT LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking(x, g_x, alpha_start, rho, gamma) #call the new function you wrote to compute the steplength
      #raise ValueError('BACKTRACKING LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 0.1
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here   
    x = np.subtract(x, np.multiply(step_length,g_x)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalgh(x) #compute gradient at new point

    #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x, k

In [44]:
my_start_x = np.array([1/512, 1/64, 1/8, 1])
my_tol= 1e-10

#exact line search
x_opt, iterations_using_exact_line_search = find_minimizer_for_four_vars(my_start_x, my_tol, EXACT_LINE_SEARCH)
print("Using exact Line Search: \nMinimizer: {}, Minimum function value: {}, No. of Iterations: {}\n".format((x_opt[0], x_opt[1], x_opt[2]), evalh(x_opt), iterations_using_exact_line_search))

#check what happens when you call find_minimzer using backtracking line search
x_opt_bls, iterations_using_backtracking_line_search = find_minimizer_for_four_vars(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, 1, 0.5,0.5)
print("Using Backtracking algorithm:\nMinimizer: {}, Minimum function value: {}, No. of Iterations: {}\n".format((x_opt_bls[0], x_opt_bls[1], x_opt_bls[2]), evalh(x_opt_bls), iterations_using_backtracking_line_search))


Using exact Line Search: 
Minimizer: (2.000000000138924, 4.0, 7.999999999999997), Minimum function value: 8.8565993523583e-18, No. of Iterations: 2013

Using Backtracking algorithm:
Minimizer: (1.9999999999999996, 3.999999999999993, 7.999999999999886), Minimum function value: 1.0237544252113035e-17, No. of Iterations: 37079





*   Minimizer: (2.000000000138924, 4.0, 7.999999999999997), 
*   Minimum function value: 8.8565993523583e-18





In gradient descent procedure which uses exact step length computation 
algorithm converged within $201$ iterations while in the gradient descent procedure which uses the backtracking line search procedure (with $\alpha^0 = 1, \rho = 0.5, \gamma = 0.5$) takes $37079$ iterations to converge. Here in this example, exact line search is pretty fast as compared to backtracking line search.

$\textbf{Hence we can see that similar observations hold in the $N = 4$ case as well.}$

$\textbf{Question 5}: $
$f(\textbf{x}) = f(x_1, x_2, x_3, x_4) = \frac{1}{8}(x_1 - 2)^2 + \frac{1}{64}(x_2 - 4)^2 + \frac{1}{512}(x_3 - 8)^2 + \frac{1}{4096}(x_4 - 16)^2$


The possible observations for $N > 4$ case, without actually running the program are similar like $N=3$ and $N=4$ case. For $N > 4$ also, **exact line search will be pretty fast as compared to backtracking line search and minimizer will be $(1, 2, ..., N)$ for $N$ dimentionsal function. Minimum objective function value will be $0 \hspace{3mm}\forall N \in \mathbb{N}.$**.